In [9]:
import requests

In [1]:
GENIUS_API_KEY = "Su_sv_qBB0tEdQrsGa6q-W84uzxU2zMdhuvag2nq4CLF0wrKosbhCzFMToY0nsjn"
GENIUS_SEARCH_URL = "https://api.genius.com/search"

def search_genius_songs(query):
    headers = {"Authorization": f"Bearer {GENIUS_API_KEY}"}
    params = {"q": query}
    response = requests.get(GENIUS_SEARCH_URL, headers=headers, params=params)
    if response.status_code == 200:
        return response.json()["response"]["hits"]
    return None

# query = "Shape of You Ed Sheeran"
# songs = search_genius_songs(query)
# for song in songs:
#     print(f"{song['result']['full_title']} - {song['result']['url']}")

In [2]:
!pip install beautifulsoup4


Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: /Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip


In [3]:
# from bs4 import BeautifulSoup

# def fetch_lyrics(genius_url):
#     response = requests.get(genius_url)
#     soup = BeautifulSoup(response.text, "html.parser")
#     lyrics_divs = soup.find_all("div", class_="Lyrics__Container")
#     lyrics = "\n".join([div.get_text("\n") for div in lyrics_divs])
#     return lyrics

# lyrics = fetch_lyrics(songs[0]['result']['url'])
# print(lyrics)


# import requests
# from bs4 import BeautifulSoup

# def get_genius_lyrics(url):
#     headers = {
#         "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
#     }

#     response = requests.get(url, headers=headers)
    
#     if response.status_code != 200:
#         print("Failed to retrieve the webpage")
#         return None

#     soup = BeautifulSoup(response.text, "html.parser")
    
#     # Find the div that contains the lyrics
#     lyrics_div = soup.find("div", class_="Lyrics__Container-sc-926d9e10-1 fEHzCI")
#     print("lyrics_div",lyrics_div)
#     if not lyrics_div:
#         print("Lyrics container not found")
#         return None

#     # Extract lyrics text
#     lyrics = "\n".join([line.get_text(separator="\n") for line in lyrics_div.find_all("p")])

#     return lyrics

# # Example usage
# url = "https://genius.com/Ed-sheeran-shape-of-you-lyrics"
# lyrics = get_genius_lyrics(url)
# if lyrics:
#     print(lyrics)

import requests
from bs4 import BeautifulSoup

def get_genius_lyrics(url):
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
    }

    response = requests.get(url, headers=headers)
    
    if response.status_code != 200:
        print("Failed to retrieve the webpage")
        return None

    soup = BeautifulSoup(response.text, "html.parser")
    
    # Find all divs that contain lyrics
    lyrics_divs = soup.find_all("div", {"data-lyrics-container": "true"})

    if not lyrics_divs:
        print("Lyrics container not found")
        return None

    lyrics = []
    
    for div in lyrics_divs:
        # Extract text from normal text and <a> tags
        for elem in div.children:
            if elem.name == "a":  # Extract text from links
                lyrics.append(elem.get_text(separator="\n"))
            elif elem.name is None:  # Extract plain text
                lyrics.append(elem.strip())

    return "\n".join(lyrics)

# Example usage
# url = "https://genius.com/Ed-sheeran-shape-of-you-lyrics"
# lyrics = get_genius_lyrics(url)

# if lyrics:
#     print(lyrics)


/Users/nehakolambe/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [4]:
!pip install yt-dlp

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: /Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip


In [5]:
import yt_dlp
import os
import re

def search_youtube(query):
    ydl_opts = {
        "quiet": True,
        "format": "bestaudio/best",
        "default_search": "ytsearch",
        "noplaylist": True
    }
    with yt_dlp.YoutubeDL(ydl_opts) as ydl:
        search_result = ydl.extract_info(f"ytsearch:{query}", download=False)
        if search_result and "entries" in search_result and len(search_result["entries"]) > 0:
            # return search_result["entries"][0]["webpage_url"]  # Use webpage_url for actual video URL
            first_entry = search_result["entries"][0]
            return first_entry["webpage_url"], first_entry["title"]  # Return URL and Title
    return None, None

# youtube_url = search_youtube("Shape of You Ed Sheeran")
# print(youtube_url)

In [6]:
def sanitize_filename(filename):
    """Remove invalid characters from filename"""
    return re.sub(r'[\\/*?:"<>|]', "", filename)

In [7]:
# def download_song(youtube_url, output_path="downloads/song.mp3"):
#     ydl_opts = {
#         "format": "bestaudio/best",
#         "outtmpl": output_path,
#         "postprocessors": [{
#             "key": "FFmpegExtractAudio",
#             "preferredcodec": "mp3",
#             "preferredquality": "192",
#         }],
#     }
#     with yt_dlp.YoutubeDL(ydl_opts) as ydl:
#         ydl.download([youtube_url])

# download_song(youtube_url)
# print("done")

def download_song(youtube_url, song_name):
    song_name = sanitize_filename(song_name)  # Clean up the song name for file safety
    output_path = f"downloads/{song_name}"  # Set the output filename

    ydl_opts = {
        "format": "bestaudio/best",
        "outtmpl": output_path,
        "postprocessors": [{
            "key": "FFmpegExtractAudio",
            "preferredcodec": "mp3",
            "preferredquality": "192",
        }],
    }
    with yt_dlp.YoutubeDL(ydl_opts) as ydl:
        ydl.download([youtube_url])
    
    print(f"Download complete: {output_path}")

# Test

In [8]:
query = "Love me like you do"
songs = search_genius_songs(query)
print("Link - ",songs[0]['result']['url'])
url = songs[0]['result']['url']
lyrics = get_genius_lyrics(url)
print()
if lyrics:
    print(lyrics)

youtube_url, song_name = search_youtube(query)

if youtube_url and song_name:
    if not os.path.exists("downloads"):
        os.makedirs("downloads")  # Ensure downloads directory exists

    download_song(youtube_url, song_name)
else:
    print("Song not found on YouTube.")

Link -  https://genius.com/Ellie-goulding-love-me-like-you-do-lyrics

[Produced by Max Martin & Ali Payami]
[Verse 1]
You're the light, you're the night
You're the colour of my blood
You're the cure, you're the pain
You're the only thing I wanna touch
Never knew that it could mean so much, so much
[Verse 2]
You're the fear, I don't care
'Cause I've never been so high
Follow me through the dark
Let me take you past the satellites
You can see the world you brought to life, to life
[Chorus]
So love me like you do, l-l-love me like you do
Love me like you do, l-l-love me like you do
Touch me like you do, t-t-touch me like you do
What are you waiting for?
[Verse 3]
Fading in, fading out
On the edge of paradise
Every inch of your skin
Is a holy grail I've gotta find
Only you can set my heart on fire, on fire, yeah
[Pre-Chorus]
I'll let you set the pace
'Cause I'm not thinkin' straight
My head's spinnin' around, I can't see clear no more
What are you waiting for?
[Chorus]
Love me like you do,